In [188]:
#### import tatqa_utils
import pandas as pd
import table_convert
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [8]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
#llm = OpenAI(temperature=0) 
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [90]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [198]:
def valuelist2docs(value_list):
    docs = []
    for item in value_list:
        docs.append(
            Document(
                page_content=str(item),
                #metadata={"categories": 2022, "original_col_name": col},
            ),
        )
    return docs
valuelist2docs(value_list)

[Document(metadata={}, page_content="{'number_value': 1452.4, 'scale': '', 'categories': ['Fixed Price'], 'metadata': ['2019', 'Years Ended September 30,']}"),
 Document(metadata={}, page_content="{'number_value': 1146.2, 'scale': '', 'categories': ['Fixed Price'], 'metadata': ['2018', 'Years Ended September 30,']}"),
 Document(metadata={}, page_content="{'number_value': 1036.9, 'scale': '', 'categories': ['Fixed Price'], 'metadata': ['2017', 'Years Ended September 30,']}"),
 Document(metadata={}, page_content="{'number_value': 44.1, 'scale': '', 'categories': ['Other'], 'metadata': ['2019', 'Years Ended September 30,']}"),
 Document(metadata={}, page_content="{'number_value': 56.7, 'scale': '', 'categories': ['Other'], 'metadata': ['2018', 'Years Ended September 30,']}"),
 Document(metadata={}, page_content="{'number_value': 70.8, 'scale': '', 'categories': ['Other'], 'metadata': ['2017', 'Years Ended September 30,']}"),
 Document(metadata={}, page_content="{'number_value': 1496.5, 's

In [183]:
def gen_code(llm, question, value_list):  
        #prompt = f"Generate a Python function 'run(table)' that can answer the following question using the following table! The function must return numeric float results with accuracy to two decimal places. If result value or calculation is percent calculation return format (value, 'percent'), every orher case return (value, '') . Do not generate explanation, nor example code, just the function.\n\nQuestion: {question}  \n\nTable: {table}"
        prompt = f"You are a helpful assistant to answer questions in the financial reports domain. Generate a Python function 'run(value_list)' that can answer the following question using the following JSON value list, annotated with category and metadata! The function must return a tuple (result_number, scale). result_number is numeric float with accuracy to two decimal places. Scale usually is thousand, million, billion, percent or an empty string. Always use categories to select the proper values for calculation. Do not generate explanation, nor example code, just the function. \n\nQuestion: {question}  \n\Value list: {value_list}"
        res = llm.invoke(prompt)
        code = res.content.replace('```python','').replace('```','')
        return (prompt, code)
    
def exec_code(code, value_list):  
        try: 
            loc = locals()   
            if not "run()" in code:
                exec(code + f"\nr = run({value_list})\n", globals(), loc)
            else: 
                exec(code + "\nr = run()\n", globals(), loc)
            return loc['r']
        except Exception as e:
                s = '[Error]'+ str(e)
                print(s)
                return (s,'')


In [206]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')
item = devdf.iloc[0]
table = item['table']['table']
#table = table_convert.fill_table_headers(table)
value_list = table_convert.convert_table(table)

docs = valuelist2docs(value_list)
v_vectorstore = FAISS.from_documents(docs, OpenAIEmbeddings())
v_retriever = v_vectorstore.as_retriever()
sel = v_retriever.invoke(question)
sel_txt = '[' + ','.join([i.page_content for i in sel])  + ']'
print(question)
pars = [p['text'] for p in item['paragraphs']]
question = item['questions'][5]['question']
#( prompt, code) = gen_code(llm, question, value_list)
( prompt, code) = gen_code(llm, question, sel_txt)
res = exec_code(code, value_list)
print(sel_txt, code, res,  item['questions'][5]['answer'])

What is the percentage change in Other in 2019 from 2018?
[{'number_value': 70.8, 'scale': '', 'categories': ['Other', '2017', 'Years Ended September 30,']},{'number_value': 44.1, 'scale': '', 'categories': ['Other', '2019', 'Years Ended September 30,']},{'number_value': 56.7, 'scale': '', 'categories': ['Other', '2018', 'Years Ended September 30,']},{'number_value': 1036.9, 'scale': '', 'categories': ['Fixed Price', '2017', 'Years Ended September 30,']}] 
def run(value_list):
    value_2018 = 0
    value_2019 = 0
    
    for item in value_list:
        if 'Other' in item['categories']:
            if item['categories'][1] == '2018':
                value_2018 = item['number_value']
            elif item['categories'][1] == '2019':
                value_2019 = item['number_value']
    
    percentage_change = ((value_2019 - value_2018) / value_2018) * 100
    
    return (round(percentage_change, 2), 'percent')
 (-22.22, 'percent') -22.22


In [208]:
from tqdm.notebook import tqdm as log_progress

res = []
for i, item in log_progress(devdf.iterrows()):
    table = item['table']['table']
    value_list = table_convert.convert_table(table)
    
    docs = valuelist2docs(value_list)
    v_vectorstore = FAISS.from_documents(docs, OpenAIEmbeddings())
    v_retriever = v_vectorstore.as_retriever()
            
    #pars = [p['text'] for p in item['paragraphs']]
    #table = transform_elements(table, remove_thousand_separators)
    #table = [remove_thousand_separators(cell) for row in table for cell in row]
    #table = [cell.replace('$', '') for row in table for cell in row]
    #print (table)
    for q in item['questions']:        
        if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']:            
            sel = v_retriever.invoke(question)
            sel_txt = '[' + ','.join([i.page_content for i in sel])  + ']'

            ( prompt, code)  = gen_code(llm, q['question'], sel_txt)
            r = exec_code(code, value_list)
            err =  None
            if r is None:
                (pred_value, pred_scale) = ("", "")
            else:   
                if len(r) == 2:                    
                    (pred_value, pred_scale) = r    
                    if isinstance(pred_value, tuple) and len(pred_value) == 2:
                        print('$$$$')
                        (pred_value, pred_scale) = pred_value                     
                    if pred_scale == "%" or pred_scale == "percentage"  :
                        pred_scale = 'percent'
                    if pred_scale not in ["", 'thousand', 'million', 'billion', 'percent']:
                        print('Invalid ', pred_scale)
                        pred_scale = ""    
                    #if item['table']['uid'] in res_scale and pred_scale != 'percent':
                    #    pred_scale = res_scale[item['table']['uid']]
                    if isinstance(pred_value, str):
                        #print("string")
                        if  pred_value.startswith('[Error]'):
                            (pred_value, pred_scale) = ("", "")
                            err,_ = r                                                        
                        
                elif len(r) == 1:
                    (pred_value, pred_scale) = (r, "")        
                else:
                    (pred_value, pred_scale) = ("", "")        
                
                
                
            #res.append({"table":table, "q":q, "pred":pred, "code": code})
            res.append(({"answer_type":q["answer_type"], "answer": q["answer"], 'scale': q["scale"]}, pred_value, pred_scale, q, code, prompt,value_list, err))


0it [00:00, ?it/s]

[Error]float division by zero


UnboundLocalError: local variable 'first_value_row_idx' referenced before assignment

In [209]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=False)
print( pred_em, pred_f1, scale_score)

0.3076923076923077 0.3076923076923077 0.8461538461538461


In [210]:
good = 0
for ans, pred, pred_scale, q,code, _,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    
    if pred  is None or isinstance(pred, tuple) or isinstance(pred, str):
        #if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
       
        #type error
        #if isinstance(pred, str):
        #    print (pred, pred_scale, ' -- ' , ans['answer'])
        #if isinstance(pred, tuple):
        #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        #print('===============================================================================')
        print('{',pred, '}', ans['answer'], ans['scale'], err)
        continue
    
    #pred = round(pred, 2)
    
    if (pred > 0 and llimit < pred and pred < ulimit) or (pred < 0 and llimit > pred and pred > ulimit) or pred==ulimit or pred == llimit:
        good = good + 1
        #if ans['scale'] == pred_scale and  pred_scale == 'thousand':
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' == ' , ans['answer'], ans['scale'])
        
        #if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
        #if  ans['scale'] == 'thousand':
        #    print (q['uid'], pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
          
    else:
        #value error
        #  if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
        #if ans['answer'] != pred and ans['scale'] == pred_scale and pred_scale != "":
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
       
        #None
        print (q['uid'], pred, pred_scale, ' <> ' , ans['answer'], ans['scale'])
        print(q['question'])
        print(value_list)
        print(code)
        
    #if isinstance(pred, str):
    #    print (pred, pred_scale, ' -- ' , ans['answer'])
    #if isinstance(pred, tuple):
    #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        
print (good/len(res))

b2786c1a-37de-4120-b03c-32bf5c81f157 -540.0 million  <>  -94 million
What was the change in the amount for Appliances in 2019 from 2018?
[{'number_value': 5686.0, 'scale': '', 'categories': ['Automotive', '(in millions)', '2019', 'Fiscal']}, {'number_value': 6092.0, 'scale': '', 'categories': ['Automotive', '(in millions)', '2018', 'Fiscal']}, {'number_value': 5228.0, 'scale': '', 'categories': ['Automotive', '(in millions)', '2017', 'Fiscal']}, {'number_value': 1221.0, 'scale': '', 'categories': ['Commercial transportation', '(in millions)', '2019', 'Fiscal']}, {'number_value': 1280.0, 'scale': '', 'categories': ['Commercial transportation', '(in millions)', '2018', 'Fiscal']}, {'number_value': 997.0, 'scale': '', 'categories': ['Commercial transportation', '(in millions)', '2017', 'Fiscal']}, {'number_value': 914.0, 'scale': '', 'categories': ['Sensors', '(in millions)', '2019', 'Fiscal']}, {'number_value': 918.0, 'scale': '', 'categories': ['Sensors', '(in millions)', '2018', 'Fisca

In [168]:
value_list = [{'number_value': 4.0, 'scale': '%', 'categories': ['Discount rate'], 'metadata': ['2019', 'September 30,', 'Domestic']}, {'number_value': 3.75, 'scale': '%', 'categories': ['Discount rate'], 'metadata': ['2018', 'September 30,', 'Domestic']}, {'number_value': 1.9, 'scale': '%', 'categories': ['Discount rate'], 'metadata': ['2019', 'September 30,', 'International']}, {'number_value': 2.8, 'scale': '%', 'categories': ['Discount rate'], 'metadata': ['2018', 'September 30,', 'International']}, {'number_value': 3.4, 'scale': '%', 'categories': ['Expected return on plan assets'], 'metadata': ['2019', 'September 30,', 'International']}, {'number_value': 3.7, 'scale': '%', 'categories': ['Expected return on plan assets'], 'metadata': ['2018', 'September 30,', 'International']}]

def run(value_list):
    domestic_2019 = None
    domestic_2018 = None
    international_2019 = None
    international_2018 = None
    
    for value in value_list:
        if 'Discount rate' in value['categories']:
            if '2019' in value['metadata'] and 'Domestic' in value['metadata']:
                domestic_2019 = value['number_value']
            elif '2018' in value['metadata'] and 'Domestic' in value['metadata']:
                domestic_2018 = value['number_value']
            elif '2019' in value['metadata'] and 'International' in value['metadata']:
                international_2019 = value['number_value']
            elif '2018' in value['metadata'] and 'International' in value['metadata']:
                international_2018 = value['number_value']
    print()
    domestic_diff = abs(domestic_2019 - domestic_2018)
    international_diff = abs(international_2019 - international_2018)
    
    return (round(abs(domestic_diff - international_diff), 2), '')

run(value_list)


(0.65, '')

In [159]:
wrong_indexes = [idx for idx, m in enumerate(metrics._details) if m['f1']<0.2]
resdf = pd.DataFrame(res)
resdf2 = resdf[resdf.index.isin(wrong_indexes)]

resdf2["answer"] = resdf2[0].apply(lambda x: x['answer'])
resdf2["scale"] = resdf2[0].apply(lambda x: x['scale'])
resdf2["qid"] = resdf2[3].apply(lambda x: x['uid'])
resdf2["q"] = resdf2[3].apply(lambda x: x['question'])
resdf2

/tmp/ipykernel_290687/596054401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resdf2["answer"] = resdf2[0].apply(lambda x: x['answer'])
/tmp/ipykernel_290687/596054401.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resdf2["scale"] = resdf2[0].apply(lambda x: x['scale'])
/tmp/ipykernel_290687/596054401.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

,0,1,2,3,4,5,6,7,answer,scale,qid,q
0,"{'answer_type': 'arithmetic', 'answer': -12.6,...",,,{'uid': 'eb787966-fa02-401f-bfaf-ccabf3828b23'...,def run(value_list):\n value_dict = {}\n ...,Generate a Python function 'run(value_list)' t...,"[{'number_value': 1452.4, 'scale': '', 'catego...",[Error]KeyError('Other'),-12.6,million,eb787966-fa02-401f-bfaf-ccabf3828b23,What is the change in Other in 2019 from 2018?
7,"{'answer_type': 'arithmetic', 'answer': 172, '...",178.67,,{'uid': 'a0414f81-8dc2-44b2-a441-2c9d9c805c4d'...,\ndef run(value_list):\n total = 0\n cou...,Generate a Python function 'run(value_list)' t...,"[{'number_value': 166.0, 'scale': '', 'categor...",None,172.0,million,a0414f81-8dc2-44b2-a441-2c9d9c805c4d,What is the 2019 average defined contribution ...
8,"{'answer_type': 'arithmetic', 'answer': 50.5, ...",40.33,,{'uid': 'bf7abd62-d9cd-48d2-8826-1457684019a3'...,\ndef run(value_list):\n total = 0\n cou...,Generate a Python function 'run(value_list)' t...,"[{'number_value': 166.0, 'scale': '', 'categor...",None,50.5,million,bf7abd62-d9cd-48d2-8826-1457684019a3,What is the 2019 average defined benefit schemes?
9,"{'answer_type': 'arithmetic', 'answer': 121.5,...",109.0,,{'uid': '4d259081-6da6-44bd-8830-e4de0031744c'...,\ndef run(value_list):\n dc_2019 = [item['n...,Generate a Python function 'run(value_list)' t...,"[{'number_value': 166.0, 'scale': '', 'categor...",None,121.5,million,4d259081-6da6-44bd-8830-e4de0031744c,What is the difference between 2019 average de...


In [161]:
resdf2

,0,1,2,3,4,5,6,7,answer,scale,qid,q
0,"{'answer_type': 'arithmetic', 'answer': -12.6,...",,,{'uid': 'eb787966-fa02-401f-bfaf-ccabf3828b23'...,def run(value_list):\n value_dict = {}\n ...,Generate a Python function 'run(value_list)' t...,"[{'number_value': 1452.4, 'scale': '', 'catego...",[Error]KeyError('Other'),-12.6,million,eb787966-fa02-401f-bfaf-ccabf3828b23,What is the change in Other in 2019 from 2018?
7,"{'answer_type': 'arithmetic', 'answer': 172, '...",178.67,,{'uid': 'a0414f81-8dc2-44b2-a441-2c9d9c805c4d'...,\ndef run(value_list):\n total = 0\n cou...,Generate a Python function 'run(value_list)' t...,"[{'number_value': 166.0, 'scale': '', 'categor...",None,172.0,million,a0414f81-8dc2-44b2-a441-2c9d9c805c4d,What is the 2019 average defined contribution ...
8,"{'answer_type': 'arithmetic', 'answer': 50.5, ...",40.33,,{'uid': 'bf7abd62-d9cd-48d2-8826-1457684019a3'...,\ndef run(value_list):\n total = 0\n cou...,Generate a Python function 'run(value_list)' t...,"[{'number_value': 166.0, 'scale': '', 'categor...",None,50.5,million,bf7abd62-d9cd-48d2-8826-1457684019a3,What is the 2019 average defined benefit schemes?
9,"{'answer_type': 'arithmetic', 'answer': 121.5,...",109.0,,{'uid': '4d259081-6da6-44bd-8830-e4de0031744c'...,\ndef run(value_list):\n dc_2019 = [item['n...,Generate a Python function 'run(value_list)' t...,"[{'number_value': 166.0, 'scale': '', 'categor...",None,121.5,million,4d259081-6da6-44bd-8830-e4de0031744c,What is the difference between 2019 average de...
